# Training a transformer model

In [ ]:
checkpoint = 'distilbert-base-uncased'

## Loading Tokenizer

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

## Dataset Processing

### Dataset Loading

In [ ]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding

data_files = {'train':'./../data/FiQA_and_Financial_PhraseBank_in_1/train.csv', 
              'dev':'./../data/FiQA_and_Financial_PhraseBank_in_1/dev.csv', 
              'test':'./../data/FiQA_and_Financial_PhraseBank_in_1/test.csv'}
raw_dataset = load_dataset("csv", data_files=data_files)

label2id = {
    'positive': 0,
    'neutral': 1,
    'negative': 2
}
def label_mapper(example):
    return {"Sentiment": label2id[example["Sentiment"]]}

def tokenize_function(example):
    return tokenizer(example["Sentence"], truncation=True)

raw_dataset = raw_dataset.map(label_mapper)
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
tokenized_dataset["train"][0]

In [ ]:
tokenized_dataset

### Initialising Data Collator

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Preparing data for training

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(["Sentence"])
tokenized_dataset = tokenized_dataset.rename_column("Sentiment", "labels")
tokenized_dataset.set_format("torch")
tokenized_dataset["train"].column_names

In [ ]:
train_dataloader = DataLoader(
                        tokenized_dataset["train"],
                        shuffle=True,
                        batch_size=16,
                        collate_fn=data_collator
                    )

In [ ]:
dev_dataloader = DataLoader(
                        tokenized_dataset["dev"],
                        batch_size=16,
                        collate_fn=data_collator
                    )

In [ ]:
test_dataloader = DataLoader(
                        tokenized_dataset["test"],
                        batch_size=16,
                        collate_fn=data_collator
                    )

## Model Training

### Loading Model

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
model

### Initialising Optimizer

In [ ]:
from transformers import AdamW

In [ ]:
optimizer = AdamW(model.parameters(), lr=3e-5)

### Initialising Scheduler

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

### training Loop

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

device

In [ ]:
def train(model):
    progress_bar = tqdm(range(len(train_dataloader)))
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        output = model(**batch)
        loss = output.loss
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        progress_bar.update(1)

In [ ]:
import evaluate
import numpy as np

load_accuracy = evaluate.load("accuracy")
load_f1 = evaluate.load("f1")

def evaluate(model, test_dataloader):
    progress_bar = tqdm(range(len(test_dataloader)))
    model.eval()
    for batch in test_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
    
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        load_accuracy.add_batch(predictions=predictions, references=batch["labels"])
        load_f1.add_batch(predictions=predictions, references=batch["labels"])
        progress_bar.update(1)
    
    print(load_accuracy.compute())
    print(load_f1.compute(average='macro'))

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_epochs))

for epoch in range(num_epochs):
    train(model)
    evaluate(model, dev_dataloader)
    lr_scheduler.step()
    progress_bar.update(1)

Inside the training loop, optimization happens in three steps:
* Backpropagate the prediction loss with a call to `loss.backward()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.
* Once we have our gradients, we call `optimizer.step()` and `lr_sceduler.step()` to adjust the parameters by the gradients collected in the backward pass.
* Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.

## Saving the model

In [ ]:
model.save_pretrained('./../../../model/fin_sentiment_distilbert/')

## Model Evaluation

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('./../../../model/fin_sentiment_distilbert/')
evaluate(model, test_dataloader)